# Akari(aka: Light Up! 照明🔦)

- 在空白格子里放置灯，灯可以照亮横纵所有行，除非遇到黑色格子的阻碍；
- 黑色格子里的数字表示这个黑色格子上下左右四个位置的灯的数量

输入数据：棋盘（用W表示白色格子，B表示黑色格子，数字表示“既是黑色格子又是数字”的格子，棋盘的长、宽；

输出数据：放置的情况

- 约束条件：
    - 数字格的灯约束必须满足；
    - 对每个垂直 / 水平的连续白色格形成的宫，里面的灯泡不能超过1（不然就会存在互相照亮这种情况）
    - 对每个空白格子，搜索上下左右四个方向以及自己的位置（直到在所有方向都到达边界或者遇到黑色格子），这么多格子里必须有至少一盏灯（这个约束限制了每个白色格子都必须被照亮）

-------


- Place lights in the blank squares. Lights illuminate all rows and columns horizontally and vertically, unless obstructed by black squares.
The numbers in black squares indicate the number of lights in the four positions surrounding the black square: up, down, left, and right.
Input data: The chessboard (represented by 'W' for white squares, 'B' for black squares, and numbers indicating squares that are both black and numbered), as well as the length and width of the chessboard.

- Output data: The placement situation.

- Constraints:

    - **The light constraints for numbered squares must be satisfied**.
    - **For each vertical/horizontal continuous group of white squares forming a block, there cannot be more than one light bulb inside** (to avoid situations where lights illuminate each other).
    - For each blank square, search in the up, down, left, and right directions, as well as its own position (until reaching the boundary in all directions or encountering a black square). There must be at least one light among these squares (this constraint ensures that every white square must be illuminated).

![](https://cdn.jsdelivr.net/gh/SmilingWayne/picsrepo/202502080123616.png)

![](https://cdn.jsdelivr.net/gh/SmilingWayne/picsrepo/202502080123238.png)


In [2]:
def readGrid(path):
    with open(f"../assets/data/Akari/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res
# display input data 
if __name__  == "__main__":
    m, n, grid = readGrid("8_14x24")
    print(m, n)
    for g in grid:
        print(g)

14 24
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', 'x', '-', '1', '-', 'x', '-', '-', '-', '-', '0', '-', '-', 'x', '-', '-', '-', '-', 'x', '-', 'x', '-', '1', '-']
['-', '-', 'x', '-', '-', 'x', '-', '0', 'x', '-', 'x', '-', '-', 'x', '-', '2', '1', '-', 'x', '-', '-', 'x', '-', '-']
['-', '0', '-', 'x', '-', 'x', '-', '-', '-', '-', '0', '-', '-', 'x', '-', '-', '-', '-', 'x', '-', 'x', '-', 'x', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '1', 'x', 'x', '-', '-', '-', '-', '1', '-', '-', '-', '-', 'x', '-', '-', '-', '-', '2', '-', 'x', 'x', 'x', '-']
['-', '-', '-', '-', '-', '-', '-', 'x', '-', '-', '-', '-', '2', '-', '-', '-', '-', '3', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', 'x', '-', '-', '-', '-', '1', '-', '-', '-', '-', 'x', '-', '-', '-', '-', '-', '-', '-']
['-', '0', '1', '0', '-', 

In [ ]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp


def akari_solver(X, Y, grid):
    
    # try:
    #     len(grid) == X * Y
    # except Exception:
    #     raise(f"Fatal: Check length of grid != {X}*{Y}")
    
    def akari_cross_neighbours(pos_x, pos_y):
    # Cross site
        res = []
        if pos_x + 1 < X and grid[pos_x + 1][pos_y] in "-":
            res.append(x[pos_x + 1, pos_y])
        
        if pos_x - 1 >= 0 and grid[pos_x - 1][pos_y] in "-":
            res.append(x[pos_x - 1, pos_y])
        
        if pos_y + 1 < Y and grid[pos_x][pos_y + 1] in "-":
            res.append(x[pos_x, pos_y + 1])
        
        if pos_y - 1 >= 0 and grid[pos_x][pos_y - 1] in "-":
            res.append(x[pos_x, pos_y - 1])

        return res
    
    def akari_neighbours(pos_x , pos_y):
        result = []
        ticks = 1
        while pos_y + ticks < Y and grid[pos_x][pos_y + ticks] in "-":
            result.append(x[pos_x, pos_y + ticks])
            ticks += 1
        ticks = 1
        while pos_y - ticks >= 0 and grid[pos_x][pos_y - ticks] in "-":
            result.append(x[pos_x, pos_y - ticks])
            ticks += 1
        ticks = 1
        while pos_x - ticks >= 0 and grid[pos_x - ticks][pos_y] in "-":
            result.append(x[pos_x - ticks, pos_y])
            ticks += 1
        ticks = 1
        while pos_x + ticks < X and grid[pos_x + ticks][pos_y] in "-":
            result.append(x[pos_x + ticks, pos_y])
            ticks += 1
        
        result.append(x[pos_x, pos_y])
        return result

    model = cp.CpModel()
    x = {}
    
    for i in range(X):
        for j in range(Y):
            if grid[i][j] in "-":
                x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
            else:
                x[i, j] = 0
            
    for i in range(X):
        tmp_cabin = []
        for j in range(Y):
            if grid[i][j] in "-":
                tmp_cabin.append(x[i, j])
                if j == Y - 1:
                    model.Add(sum(tmp_cabin) <= 1)
            else:
                if len(tmp_cabin) > 0:
                    model.Add(sum(tmp_cabin) <= 1)
                tmp_cabin = []
                if grid[i][j] in "0123456789":
                    light_constr = akari_cross_neighbours(i, j)
                    model.Add(sum(light_constr) == int(grid[i][j]))
                
    for j in range(Y):
        tmp_cabin = []
        for i in range(X):
            if grid[i][j] in "-":
                tmp_cabin.append(x[i, j])
                if i == X - 1:
                    model.Add(sum(tmp_cabin) <= 1)
            else:
                if len(tmp_cabin) > 0:
                    model.Add(sum(tmp_cabin) <= 1)
                tmp_cabin = []
    
    for i in range(X):
        for j in range(Y):
            if grid[i][j] in "-":
                tobe_lighted = akari_neighbours(i, j)
                model.Add(sum(tobe_lighted) >= 1)
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(X):
            for j in range(Y):
                if grid[i][j] == "x":
                    print("B", end=" ")
                elif grid[i][j] in "0123456789":
                    print(grid[i][j], end=" ")
                else:
                    if solver.Value(x[i, j]) > 1e-3:
                        print("*", end = " ")
                    else:
                        print("-", end = " ")
            print()
        print()
        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
        
if __name__ == "__main__":
    
    m, n, grid = readGrid("8_14x24")
    akari_solver(m, n, grid)

- - - - - - - - - - - - - - - - - - - - - * - - 
- B * 1 - B - - * - 0 - - B - - * - B - B - 1 - 
* - B - * B - 0 B - B * - B * 2 1 - B - * B * - 
- 0 - B - B - - * - 0 - - B - * - - B - B * B - 
- - * - - - - - - - - - - - - - - - - - - - - - 
- 1 B B - - * - 1 - - - * B - - - * 2 - B B B - 
- * - - - - - B * - - - 2 * - - - 3 * - - - - - 
- - - - - * B * - - - 1 - - - - B * - - - - - - 
- 0 1 0 - B * - - - B * - - - 1 * - - - B B B - 
- - * - - - - - - - - - - - - - - - - - - - - - 
- B - B - 0 - - - - B - - B - * - - B - 1 * B - 
- * 2 * - B - 1 1 * 1 - * 1 - 2 B - B * - B - * 
- 1 - B - 0 - * - - B - - B - * - - B - B * 1 - 
- - * - - - - - - - - - - - - - - - - - - - - - 

NumConflicts: 0
NumBranches: 0
WallTime: 0.0034500000000000004


> Special Thanks to This Bilibili UP [This video](https://www.bilibili.com/video/BV1nE411s7cx/)
>
> And [This Github repo](https://github.com/Borroot/akari)'s help.
